<a href="https://colab.research.google.com/github/Mrajie88/-/blob/master/competition_continue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import time
import torch
from mpl_toolkits.mplot3d import Axes3D
import os

In [0]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
Tesla P4


In [0]:
path = "/content/drive/My Drive/"
train_df_source = pd.read_csv(path+'train.csv')
test_df_source = pd.read_csv(path+'test.csv')
event_df_source = pd.read_csv(path+'event.csv')

In [0]:
def f_eng(df, event_df, is_train=True):
    df = pd.merge(df, event_df.loc[:, ['event_id', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc']], how='left',
                  on='event_id')
    # df['dis'] = np.sqrt(df['x']**2+df['y']**2+df['t']**2)
    # 时间差
    df['t_o']=df['t']-df['terror']

    df['q_mean'] = df['q']-df['q'].groupby(df['event_id']).transform(np.mean)
    df['t_mean'] = df['t']-df['t'].groupby(df['event_id']).transform(np.mean)
    df['t_std'] = df["t"].groupby(df["event_id"]).transform(np.std)
    df['t_ptp'] = df["t"].groupby(df["event_id"]).transform(np.ptp)
    df["x_xcmc"] = df["x"]-df["xcmc"]
    df["y_ycmc"] = df["y"] - df["ycmc"]
    df["dis"] = np.sqrt(df["x_xcmc"] ** 2+df["y_ycmc"] ** 2 +df["t"] ** 2)
    
    if is_train:
        df = df.drop('flag', axis=1)
    df = df.drop(['hit_id', 'z', 'event_id'], axis=1)
    return df

In [0]:
labels = train_df_source['flag']
train_df = f_eng(train_df_source, event_df_source)
test_df = f_eng(test_df_source, event_df_source, False)
print(train_df.columns.values.tolist())
# 小样本训练
train_df = train_df.iloc[:10000000, :]
labels = labels[:10000000]

train_x, val_x, train_y, val_y = train_test_split(train_df, labels, test_size=0.33, random_state=42)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


['x', 'y', 't', 'terror', 'q', 'energymc', 'thetamc', 'phimc', 'xcmc', 'ycmc', 't_o', 'q_mean', 't_mean', 't_std', 't_ptp', 'x_xcmc', 'y_ycmc', 'dis']


In [0]:
fea_imp_list = []
params = {
    "task":"train",
    "boosting_type":"gbdt",
    "metric":"auc",
    "num_leaves":255,
    "learning_rate":0.01,
    "feature_fraction":0.9,
    "bagging_fraction":0.8,
    "verbose":50
}

In [0]:
batch_size = 2000000
gbm = None
max_len = 0
for i in range(0,len(train_x),batch_size):
  if i+ batch_size >len(train_x):
    max_len = batch_size
  else:
    max_len = len(train_x)-i-1
  print("=========================================================数据准备======================================================================================")
  lgb_train = lgb.Dataset(train_x.iloc[i:i+max_len,:],train_y.iloc[i:i+max_len])
  lgb_test = lgb.Dataset(val_x,val_y,reference = lgb_train)
  print("=========================================================模型训练======================================================================================")
  gbm = lgb.train(params,lgb_train,num_boost_round=10000,init_model = gbm,valid_sets = lgb_test,early_stopping_rounds = 2000,keep_training_booster=True)
  gbm.save_model("/content/drive/My Drive/lgb_model.txt")
  print("============================================================模型训练完成===========================================================================================")
fea_imp_list.append(gbm.feature_importances_)

流式输出内容被截断，无法输出最后 5000 行内容。
[15024]	valid_0's auc: 0.996525
[15025]	valid_0's auc: 0.996525
[15026]	valid_0's auc: 0.996525
[15027]	valid_0's auc: 0.996525
[15028]	valid_0's auc: 0.996525
[15029]	valid_0's auc: 0.996525
[15030]	valid_0's auc: 0.996525
[15031]	valid_0's auc: 0.996525
[15032]	valid_0's auc: 0.996525
[15033]	valid_0's auc: 0.996525
[15034]	valid_0's auc: 0.996525
[15035]	valid_0's auc: 0.996525
[15036]	valid_0's auc: 0.996525
[15037]	valid_0's auc: 0.996525
[15038]	valid_0's auc: 0.996525
[15039]	valid_0's auc: 0.996525
[15040]	valid_0's auc: 0.996525
[15041]	valid_0's auc: 0.996525
[15042]	valid_0's auc: 0.996525
[15043]	valid_0's auc: 0.996525
[15044]	valid_0's auc: 0.996525
[15045]	valid_0's auc: 0.996525
[15046]	valid_0's auc: 0.996525
[15047]	valid_0's auc: 0.996525
[15048]	valid_0's auc: 0.996525
[15049]	valid_0's auc: 0.996525
[15050]	valid_0's auc: 0.996525
[15051]	valid_0's auc: 0.996525
[15052]	valid_0's auc: 0.996525
[15053]	valid_0's auc: 0.996525
[15054]	valid

In [0]:
t = time.time()
test_pre = pd.DataFrame(gbm.predict(test_df)[:, 1], columns=['flag_pred'])
print('test_pre runtime:', time.time() - t)
sub = pd.concat([test_df_source['hit_id'], test_pre,test_df_source['event_id']], axis=1)
fea_imp_list.append(clf.feature_importances_)
print('runtime:', time.time() - t)

test_pre runtime: 29222.276727199554
runtime: 29222.325490951538


In [0]:
# 特征重要性
fea_imp_dict = dict(zip(train_df.columns.values, np.mean(fea_imp_list, axis=0)))
fea_imp_item = sorted(fea_imp_dict.items(), key=lambda x: x[1], reverse=True)
for f, imp in fea_imp_item:
    print('{} = {}'.format(f, imp))

t_mean = 256190.5
x = 243347.5
q = 232606.0
y = 227337.0
xcmc = 197625.5
thetamc = 175456.0
ycmc = 166372.0
t = 158382.0
q_mean = 157676.0
t_std = 142223.0
energymc = 130269.0
t_ptp = 123347.0
t_o = 119587.5
terror = 116736.5
phimc = 92844.5


In [0]:
# sub.to_csv('drive/My Drive/sub_prob_{}_{}_{}.csv'.format(best_auc, best_f1,sub['flag_pred'].mean()),index=False)
sub['flag_pred'] = sub['flag_pred'].apply(lambda x: 1 if x >= best_t else 0)
sub.to_csv('drive/My Drive/submition_{}_{}.csv'.format(time.asctime( time.localtime(time.time()) ),sub['flag_pred'].mean()),index=False)

print('runtime:', time.time() - t)
print('finish.')

runtime: 29252.052520036697
finish.
